# dd_limit_plot
is an attempt to make a set of python direct detection result plotting conveniences. 

Results-- whether they be sensitivities, upper limits etc. are .csvs (as bunch are stored in ../data/ ) 
In addition, you can optionally add properties (labels, plot colors, bibliography information) for a .csv file named FILENAME.csv under the entry for "FILENAME" in ../data/result_metadata.toml

The script will look in ../data/ by default, and you can provide just the file name (even including wildcards) to DD_result (if there are several matches, you'll get an error + a list of available matches) 

In [ ]:
from dd_limit_plot import DD_result, find_dd_results
import matplotlib.pyplot as plt

## Basic example
Let us plot the XENONnT and LZ spin-independent limits, as well as just an example .csv file called example_file.csv:  

In [ ]:
xenon_limit = DD_result("WIMPSI_XENONnT_2023_2303.14729.csv")
lz_limit    = DD_result("WIMPSI_LZ_2023_2207.03764.csv")
example_line = DD_result("example_*.csv")


plt.clf()
xenon_limit.plot_upper_limit(color="blue", label="XENON")
lz_limit.plot_upper_limit(color="orange", label="LZ")
example_line.plot_upper_limit(color="magenta", label="example")

plt.legend()
plt.yscale("log")
plt.xscale("log")
plt.xlabel("WIMP Mass [GeV$/c^2$]")
plt.ylabel("WIMP-nucleon cross-section [cm$^2$]")
plt.show()


### useful things
The Xenon and LZ lines have bibliographical information included in their metadata, so you can print this out: 

In [ ]:
print(xenon_limit.reference)
print(lz_limit.reference)

### Useful things 2

The Xenon and LZ results don't just come with the default wimp_mass and upper_limits fields-- they also include the experiment sensitivity-- you can plot the sensitivity lines and bands:

In [ ]:
plt.clf()
xenon_limit.plot_band("sensitivity_m2sigma","sensitivity_p2sigma", alpha=0.15,color="blue") #between -2 sigma and +2sigma
xenon_limit.plot_band("sensitivity_m1sigma","sensitivity_p1sigma", alpha=0.3,color="blue") #between -1 sigma and +1sigma
xenon_limit.plot_sensitivity(color="blue")
lz_limit.plot_band("sensitivity_m2sigma","sensitivity_p2sigma", alpha=0.15,color="orange") #between -2 sigma and +2sigma
lz_limit.plot_band("sensitivity_m1sigma","sensitivity_p1sigma", alpha=0.3,color="orange") #between -1 sigma and +1sigma
lz_limit.plot_sensitivity(color="orange")
xenon_limit.plot_upper_limit(color="blue", label="XENON")
lz_limit.plot_upper_limit(color="orange", label="LZ")


plt.legend()
plt.yscale("log")
plt.xscale("log")
plt.xlabel("WIMP Mass [GeV$/c^2$]")
plt.ylabel("WIMP-nucleon cross-section [cm$^2$]")
plt.show()

## Finding multiple results


   By using the find_dd_results, you can get an array of dd_results matching a string-- for this reason, we recommend to name new files following the pattern you see in data-- targetsignal_experiment_year_morename.csv 
   
as an example, lets plot all xenon SI wimp results: @


In [ ]:
xenon_limit.plot_upper_limit

In [ ]:
xenon_results = find_dd_results("WIMPSI_XENON*")

plt.clf()
for k,result in xenon_results.items(): 
    try: #some of the files are projections that do not have upper limits
        result.plot_upper_limit(label=result.label)
    except: 
        result.plot_sensitivity(label=result.label)


plt.legend()
plt.yscale("log")
plt.xscale("log")
plt.xlabel("WIMP Mass [GeV$/c^2$]")
plt.ylabel("WIMP-nucleon cross-section [cm$^2$]")
plt.show()



## Plotting many results with nice labels
if you have https://github.com/cphyc/matplotlib-label-lines installed, you can have all plot labels neatly placed alongside your lines, with the option to manually choose the x-position in order to show them all: 

In [ ]:
all_wimp_limits = find_dd_results("WIMPSI*")
nufloor = DD_result("nufloor*")

In [ ]:
fig = plt.subplots(figsize=(10,5), dpi=150)

for k,i in all_wimp_limits.items():
    try:
        i.plot_upper_limit(label=i.label)
    except:
        i.plot_sensitivity(label=i.label)
plt.xlim([1e-1, 1e3])
plt.ylim([5e-48, 1e-35])

plt.yscale("log") #if you do not set hte 
plt.xscale("log")
plt.xlim([1e-1, 1e3])
plt.ylim([5e-48, 1e-35])
nufloor.plot_band(0., "nufloor", color="gray", alpha=0.5)
try:
    DD_result.add_line_legends(
        position_overrides = { #if the line labels overlap, you can set positions here: 
            #"WIMPSI_XENON1T_2019_S2onlyupper_limit":3,
            #"WIMPSI_XENON1T_2019_S2onlyMigdalupper_limit":0.2
        },
        size=6,
        xmax=1000,
        #alpha = 0.5,
    )
except: 
    #perhaps you do not have the matplotlib-label-line package?
    plt.legend()
plt.xlabel("WIMP Mass [GeV$/c^2$]")
plt.ylabel("WIMP-nucleon cross-section [cm$^2$]")
plt.xlim([0.5, 1e3])
plt.ylim([1e-50, 1e-35])
#plt.legend(loc='upper right', fontsize='small', frameon=False)
plt.show()

#### Fully adjusted:

In [ ]:
fig = plt.subplots(figsize=(10,5), dpi=150)

for k,i in all_wimp_limits.items():
    try:
        i.plot_upper_limit(label=i.label)
    except:
        i.plot_sensitivity(label=i.label)
plt.xlim([1e-1, 1e3])
plt.ylim([5e-48, 1e-35])

plt.yscale("log") #if you do not set hte 
plt.xscale("log")
plt.xlim([1e-1, 1e3])
plt.ylim([5e-48, 1e-35])
nufloor.plot_band(0., "nufloor", color="gray", alpha=0.5)
try:
    DD_result.add_line_legends(
        position_overrides = { #if the line labels overlap, you can set positions here: 
            #"WIMPSI_XENON1T_2019_S2onlyupper_limit":3,
            #"WIMPSI_XENON1T_2019_S2onlyMigdalupper_limit":0.2
            
             'WIMPSI_CRESSTIII_2019upper_limit':30,
             'WIMPSI_DAMIC_2020upper_limit':1.2,
             'WIMPSI_DarkSide50_2018_lowE_noquenchupper_limit':7,
             'WIMPSI_Darwin_2016_projection_200tysensitivity_median':10,
             'WIMPSI_LUX_2017_run4combinedupper_limit':40,
             'WIMPSI_LZ_2019_projectionsensitivity_median':20,
             'WIMPSI_LZ_2023_2207.03764upper_limit':700,
             'WIMPSI_PANDAXII_2020_2007.15469upper_limit':600,
             'WIMPSI_PandaX-4T_2021_2107.13438upper_limit':300,
             'WIMPSI_XENON1T_2018_1805.12562upper_limit':650,
             'WIMPSI_XENON1T_2019_S2onlyMigdalupper_limit':0.9,
             'WIMPSI_XENON1T_2019_S2onlyupper_limit':20,
             'WIMPSI_XENON1T_2021_2foldupper_limit':6,
             'WIMPSI_XENONnT_2020_projection20tysensitivity_median':50,
             'WIMPSI_XENONnT_2023_2303.14729upper_limit':100,
             'WIMPSI_darkside20k_2017_100tysensitivity_median':8,
        },
        size=6,
        xmax=1000,
        #alpha = 0.5,
    )
except: 
    #perhaps you do not have the matplotlib-label-line package?
    plt.legend()
plt.xlabel("WIMP Mass [GeV$/c^2$]")
plt.ylabel("WIMP-nucleon cross-section [cm$^2$]")
plt.xlim([0.5, 1e3])
plt.ylim([1e-50, 1e-35])
#plt.legend(loc='upper right', fontsize='small', frameon=False)
plt.show()